In [ ]:
!pip install datasets

In [ ]:
import json
import os
import requests
import time

data = None
with open("YOUR JSON FILE.json", "r") as training:
    data = json.load(training)

image_dir = "testing_data"

# Load dataset
dataset = data
total_images = len(dataset)
print(f"Dataset loaded with {total_images} records")

# Directory to save images
os.makedirs(image_dir, exist_ok=True)

# Initialize variables for time estimation
start_time = time.time()

# Download images with progress and time estimation
for i, data in enumerate(dataset):
    image_url = data.get("image_path")
    if not image_url:
        print(f"Skipping record {i}: No image URL found.")
        continue

    image_path = os.path.join(image_dir, f"image_{i}.png")

    try:
        # Track download progress
        response = requests.get(image_url, timeout=10)

        if response.status_code == 200 and response.content:
            with open(image_path, "wb") as f:
                f.write(response.content)

            # Calculate elapsed time and estimated time left
            elapsed_time = time.time() - start_time
            remaining_images = total_images - (i + 1)
            
            # Estimate remaining time
            if i > 0:
                time_per_image = elapsed_time / (i + 1)
                remaining_time = time_per_image * remaining_images
                minutes_left = remaining_time / 60

                # Calculate percentage of completion
                percentage = (i + 1) / total_images * 100
                print(f"Downloaded {i + 1}/{total_images} images ({percentage:.2f}%)")
                print(f"Estimated time remaining: {minutes_left:.2f} minutes")
        else:
            print(f"Skipping {image_url}: Invalid response ({response.status_code})")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading {image_url}: {e}")